In [1]:
import setup
setup.init_django()

In [2]:
from market.models import StockQuote

In [14]:
from django.db.models import (
    Avg, 
    F,
    RowRange,
    Window,
    Max,
    Min,
    ExpressionWrapper,
    DecimalField,
    Case,
    When,
    Value
)
from django.db.models.functions import TruncDate, FirstValue, Lag, Coalesce
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal
from django.db.models.functions import TruncDate
from django.db.models import F, Window, Avg, Case, When, Value, DecimalField, ExpressionWrapper
from django.db.models.functions import Lag
from django.utils import timezone
from decimal import Decimal
from datetime import timedelta

In [17]:
def calculate_rsi(ticker, period=14):
    """
    Calculate Relative Strength Index (RSI) using Django ORM.
    
    Args:
        ticker (str): Stock ticker symbol
        period (int): RSI period (default: 14)
    
    Returns:
        dict: RSI value and component calculations
    """
    end_date = timezone.now()
    start_date = end_date - timedelta(days=period * 4)
    
    # First, get the daily closing prices
    daily_prices = (
        StockQuote.objects
        .filter(company__ticker=ticker, time__range=(start_date, end_date))
        .annotate(date=TruncDate('time'))
        .values('date')
        .annotate(close_price=Avg('close_price'))
        .order_by('date')
    )
    
    # Convert to list to do calculations in Python
    prices = list(daily_prices)
    
    if len(prices) < period + 1:
        raise ValueError(f"Insufficient data points. Need at least {period + 1} days of data.")
    
    # Calculate gains and losses
    changes = []
    for i in range(1, len(prices)):
        curr_price = Decimal(str(prices[i]['close_price']))
        prev_price = Decimal(str(prices[i-1]['close_price']))
        change = curr_price - prev_price
        changes.append({
            'gain': max(change, Decimal('0')),
            'loss': max(-change, Decimal('0'))
        })
    
    # Calculate initial averages
    initial_gains = sum(c['gain'] for c in changes[:period]) / period
    initial_losses = sum(c['loss'] for c in changes[:period]) / period
    
    # Calculate EMA of gains and losses
    avg_gain = initial_gains
    avg_loss = initial_losses
    alpha = Decimal(1) / Decimal(period)
    
    # Update moving averages using EMA formula
    for change in changes[period:]:
        avg_gain = (avg_gain * (1 - alpha) + change['gain'] * alpha)
        avg_loss = (avg_loss * (1 - alpha) + change['loss'] * alpha)
    
    # Calculate final RSI
    if avg_loss == 0:
        rsi = 100
    else:
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
    
    return {
        'rsi': round(float(rsi), 4),
        'avg_gain': round(float(avg_gain), 4),
        'avg_loss': round(float(avg_loss), 4),
        'period': period
    }

In [18]:
rsi_data = calculate_rsi('AAPL')

In [19]:
rsi_data

{'rsi': 32.3708, 'avg_gain': 0.5495, 'avg_loss': 1.148, 'period': 14}